In [2]:
from pathlib import Path
import sys

import pandas as pd
from sklearn.tree import DecisionTreeClassifier

rank = 0 
sys.path[0] = f'{Path().resolve().parents[rank]}' # mymoduleが上の階層にある場合rankを調整してコメント解除
from mymodule import PipeLine, grid_serch_cv
from kayano import age_categolize, cholesterol_mean

In [4]:
###################### グリッドサーチ用のパイプラインを用意 ########################
def pipe_1(df, model, param_grid):
    print(' '*15, 'ベースライン関数')
    pipe = PipeLine()
    pipe(df)
    # 以下グリッドサーチ
    pack = pipe.fold_out_split()
    gs_model = grid_serch_cv(pack, param_grid, model)
    return gs_model

def pipe_2(df, model, param_grid):
    print(' '*15, '年齢カテゴライズ')
    pipe = PipeLine()
    pipe(df)
    pipe.df_num = age_categolize(pipe.df_num)
    # 以下グリッドサーチ
    pack = pipe.fold_out_split()
    gs_model = grid_serch_cv(pack, param_grid, model)
    return gs_model

def pipe_3(df, model, param_grid):
    print(' '*15, 'コレステロール平均補完')
    df_cho = cholesterol_mean(df)  # HeartDiseaseを参照するため前もって処理
    pipe = PipeLine()
    pipe(df_cho)  # df_choを代入
    # 以下グリッドサーチ
    pack = pipe.fold_out_split()
    gs_model = grid_serch_cv(pack, param_grid, model)
    return gs_model

def pipe_4(df, model, param_grid):
    print(' '*15, '年齢補完 and コレステロール平均補完')
    df_cho = cholesterol_mean(df)  # HeartDiseaseを参照するため前もって処理
    pipe = PipeLine()
    pipe(df_cho)  # df_choを代入
    pipe.df_num = age_categolize(pipe.df_num)
    # 以下グリッドサーチ
    pack = pipe.fold_out_split()
    gs_model = grid_serch_cv(pack, param_grid, model)
    return gs_model

In [5]:
df = pd.read_csv('./data/train.csv')
#################### グリッドサーチの探索候補を用意 #################
tree_grid = {'max_depth':[3, 4, 5, 6, 7, 8],
              'min_samples_leaf':[8,9,10,11,12,13],
              'min_samples_split':[2, 3, 4, 5],
              'random_state': [42]}

############################# 用意した関数に渡す引数 ##############################
pipe_kwargs = {'df': df,
               'model': DecisionTreeClassifier,  # sklearnのモデル
               'param_grid': tree_grid}

######################## 用意したパイプラインをfor文で訓練 #########################
pipe_lines = [pipe_1, pipe_2, pipe_3, pipe_4]  # 作ったdemo関数をリストでまとめる （）いらない
models = []  # 訓練したモデルの格納先
for pipe in pipe_lines:
    print('#'*20,f'{pipe.__name__}の結果', '#'*20)  # 表示結果のラベル
    model = pipe(**pipe_kwargs)
    models.append(model)
    print()

#################### pipe_1の結果 ####################
                     ベースライン関数
-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,0.826281,0.828571,0.849372,0.838843
test,0.797927,0.852174,0.816667,0.834043



#################### pipe_2の結果 ####################
                年齢カテゴライズ
-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,0.821826,0.873239,0.778243,0.823009
test,0.818653,0.912621,0.783333,0.843049



#################### pipe_3の結果 ####################
                コレステロール平均補完
-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,0.850780,0.901869,0.807531,0.852097
test,0.803109,0.910000,0.758333,0.827273



#################### pipe_4の結果 ####################
                年齢補完 and コレステロール平均補完
-------------------- 評価結果 --------------------


,accuracy_score,precision_score,recall_score,f1_score
train,0.850780,0.905660,0.803347,0.851441
test,0.803109,0.894231,0.775000,0.830357
